# Task 1: ANN Regression for robot arm control 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor    # multilayer perceptron for regression

In [4]:
def direct_kin_(joints, links, origin = [0, 0]):
# implement the forward kinematics for a two joints planar manipulator
# it's implemented externally so it can be used inside or outside the arm class
    X = np.zeros(3)
    Y = np.zeros(3)
    X[0] = origin[0]
    Y[0] = origin[1]
    X[1] = X[0] + links[0] * np.cos(joints[0])
    Y[1] = Y[0] + links[0] * np.sin(joints[0])
    X[2] = X[1] + links[1] * np.cos(joints[0] + joints[1])
    Y[2] = Y[1] + links[1] * np.sin(joints[0] + joints[1])
    return [X, Y]   # return the coordinates of all link endpoints

def deg2rad(degrees):
# simple function for converting degrees to radiants
    return degrees*np.pi/180

In [5]:
class arm():
### the arm class contains all the methods for defining a two joints planar manipulator,
### and implement a neural network inverse kinematics solver for it

    def __init__(self, links = [10, 10], origin = [0, 0], init = [0, 0]):
    # class contructor, defining the basic attributes of the arm and initial configuration
        self.link1 = links[0]
        self.link2 = links[1]
        self.x0 = origin[0]
        self.y0 = origin[1]
        self.joint1 = init[0]
        self.joint2 = init[1]
        self.direct_kin()

    def direct_kin(self):
    # this forward kinematic function calculate the Cartesian coordinates for the current joint configuration    
        [self.X, self.Y] = direct_kin_([self.joint1, self.joint2], [self.link1, self.link2], [self.x0, self.y0])

    def plot_arm(self):
    # 2D plot of the current arm configuration
        plt.plot([-20,20],[0,0],'k')
        plt.plot(self.X, self.Y, linewidth=2.0)
        plt.plot(self.X, self.Y, 'ro', linewidth=2.0)
        sum_links = (self.link1 + self.link2) * 1.1
        plt.axis([-sum_links, sum_links, -1, sum_links])
        plt.axis('equal')
        plt.show()

    def create_data(self, ann, n_train, n_test, range1, range2):
    # prepare the training and test sets for the neural network solver
        self.inv_solver = ann
        n_data = n_train + n_test
        joint_space = np.hstack((np.random.uniform(range1[0], range1[1], size=(n_data, 1)), np.random.uniform(range2[0], range2[1], size=(n_data,1))))
        cartesian_space = np.zeros(np.shape(joint_space))
        for i in range(len(joint_space)):
            ax, ay = direct_kin_(joint_space[i], [self.link1, self.link2])
            cartesian_space[i] = [ax[2], ay[2]]
        self.cart_train = np.asarray(cartesian_space[:n_train,:])
        self.joint_train = np.asarray(joint_space[:n_train,:])
        self.cart_test = np.asarray(cartesian_space[n_train:,:])
        self.joint_test = np.asarray(joint_space[n_train:,:])
            
    def train_inv_kin(self):
    # train the kinematic solver
        self.inv_solver.fit(self.cart_train, self.joint_train)
        score = self.inv_solver.score(self.cart_train, self.joint_train)
        return(np.mean(score)) # return training accuracy

    def test_inv_kin(self):
    # test the kinematic solver
        score = self.inv_solver.score(self.cart_test, self.joint_test)
        return(np.mean(score)) # return testing accuracy

    def inv_kin(self, Cartesian):
    # query the trained inverse kinematic solver on a single Cartesian target
        joints = self.inv_solver.predict([Cartesian])
        [self.joint1, self.joint2] = joints[0]
        self.direct_kin()
        err = np.sqrt((Cartesian[0]-self.X[2])**2+(Cartesian[1]-self.Y[2])**2)
        return(err, [self.X[2], self.Y[2]])

Task 1

A. change the network structure (number of layers and neurons), and parameters (transfer functions, learning rate, algorithms, stop conditions): how does prediction accuracy change?

B. change the quantity of training data, and the joint ranges: how does that affect accuracy?

Perform systematic tests on appropriate values and ranges (how do you choose them?) and report your results, answering the questions.

C.	Optional: Extend the code so that the ANN for inverse kinematics is able to control a 3 joint robot arm moving in the 3D space. Add the 3rd joint and the z axis to the forward kinematics equations. Extend the ANN to 3 inputs and 3 outputs, train it and analyse the learning performance.

# Your submission below

# Task 1.A: Network Structure and Parameter Variation Analysis (Changing Network Structure and Parameters)

### In this task, we explore how changing the neural network's structure (number of layers and neurons) and parameters (activation functions, learning rate, algorithms, stop conditions) affects prediction accuracy.

## Initializing the arm object

In [6]:
# Initialize the arm object
a = arm()

## Hyperparameter Configuration


In [7]:
# Define lists for different hyperparameters
hidden_layer_sizes = [(100),(100,50)]
activation_functions = ['relu', 'identity', 'tanh']
learning_rates = ['constant', 'adaptive', 'invscaling']
solvers = ['adam', 'lbfgs', 'sgd']
max_iters = [500,1000]
trial_list =[1, 2, 3]

## Train and Test Data Configuration

In [8]:

# Quantity of training data
n_train = 1800
n_test = 200

# Joint ranges
j1_range = (0, np.pi/2)
j2_range = (0, np.pi)

## Hyperparameter Tuning and Evaluation Loop


In [ ]:
# Loop through each combination of hyperparameters and trials
for hidden_size in hidden_layer_sizes:
    for activation in activation_functions:
        for solver in solvers:
            for learning_rate in learning_rates:
                for max_iter in max_iters:
                    for trial in trial_list:
                        # Initialize MLPRegressor with current hyperparameters
                        ann = MLPRegressor(hidden_layer_sizes=hidden_size, activation=activation,
                                           solver=solver, learning_rate=learning_rate,
                                           max_iter=max_iter, tol=1e-4)
                        
                        # Create training and testing data
                        a.create_data(ann, n_train, n_test, j1_range, j2_range)
                        
                        # Train the inverse kinematics solver and evaluate its performance
                        train_accuracy = a.train_inv_kin()
                        test_accuracy = a.test_inv_kin()
                        
                        # Print the results for the current trial and hyperparameters
                        print(f"Trial Number : {trial}, Hidden Layers: {hidden_size}, Activation: {activation}, Learning Rate: {learning_rate}, "
                              f"Solver: {solver}, Max Iter: {max_iter}, "
                              f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

### Observations
By varying the network structure and parameters, we observe changes in prediction accuracy. This exploration helps us understand the impact of different configurations on the performance of the neural network-based inverse kinematics solver.

# TASK 1.B: Quantity of Training Data and Joint Range Impact Analysis

## Data Configuration

### Lists for different training data quantities and joint ranges are initialized.


## Train and Test Data Configurations

In [9]:
# Define Lists
n_trains = [1000, 2000, 5000]
n_tests = [200, 500, 1000]
j1_ranges = [(-np.pi/4, np.pi/2), (-np.pi/2, np.pi/2), (0, np.pi)]
j2_ranges = [(-np.pi/4, np.pi/2), (-np.pi/2, np.pi/2), (0, np.pi)]

## Training and Testing Loop


In [ ]:
for n_train in n_trains:
    for n_test in n_tests:
        for j1_range in j1_ranges:
            for j2_range in j2_ranges:
                # Initialize MLPRegressor with fixed hyperparameters
                ann = MLPRegressor(hidden_layer_sizes=(100,50),
                                   activation='relu', solver='adam', learning_rate='constant',
                                   max_iter=2000, tol=1e-4)
                
                # Create training and testing data with current settings
                a.create_data(ann, n_train, n_test, j1_range, j2_range)
                
                # Train the inverse kinematics solver and evaluate its performance
                train_accuracy = a.train_inv_kin()
                test_accuracy = a.test_inv_kin()
                
                # Print the results for the current configuration
                print(f"Training Samples: {n_train}, Test Samples: {n_test}, "
                      f"Joint 1 Range: {j1_range}, Joint 2 Range: {j2_range}, "
                      f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")


# TASK 1.C: Extension to 3-Joint Robot Arm Control in 3D Space

## Updating Forward Kinematics Function
The forward kinematics function is modified to accommodate a three-joint spatial manipulator. It calculates the Cartesian coordinates of all link endpoints based on the joint angles and link lengths.



In [ ]:
def direct_kin_(joints, links, origin = [0, 0, 0]):
    # Implement the forward kinematics for a three-joint spatial manipulator
    X = np.zeros(4)
    Y = np.zeros(4)
    Z = np.zeros(4)
    X[0] = origin[0]
    Y[0] = origin[1]
    Z[0] = origin[2]
    X[1] = X[0] + links[0] * np.cos(joints[0])
    Y[1] = Y[0] + links[0] * np.sin(joints[0])
    Z[1] = Z[0]
    X[2] = X[1] + links[1] * np.cos(joints[0] + joints[1])
    Y[2] = Y[1] + links[1] * np.sin(joints[0] + joints[1])
    Z[2] = Z[1]
    X[3] = X[2] + links[2] * np.cos(joints[0] + joints[1] + joints[2])
    Y[3] = Y[2] + links[2] * np.sin(joints[0] + joints[1] + joints[2])
    Z[3] = Z[2] + links[2] * np.sin(joints[2])
    return [X, Y, Z]  # Return the coordinates of all link endpoints

def deg2rad(degrees):
    # Simple function for converting degrees to radians
    return degrees * np.pi / 180

## Updating Class Arm Definition

In [ ]:
class arm():
    ### The arm class contains all the methods for defining a three-joint spatial manipulator,
    ### and implement a neural network inverse kinematics solver for it

    def __init__(self, links = [10, 10, 10], origin = [0, 0, 0], init = [0, 0, 0]):
        # Class constructor, defining the basic attributes of the arm and initial configuration
        self.link1 = links[0]
        self.link2 = links[1]
        self.link3 = links[2]
        self.x0 = origin[0]
        self.y0 = origin[1]
        self.z0 = origin[2]
        self.joint1 = init[0]
        self.joint2 = init[1]
        self.joint3 = init[2]
        self.direct_kin()

    def direct_kin(self):
        # This forward kinematic function calculates the Cartesian coordinates for the current joint configuration
        [self.X, self.Y, self.Z] = direct_kin_([self.joint1, self.joint2, self.joint3], [self.link1, self.link2, self.link3], [self.x0, self.y0, self.z0])

    def create_data(self, ann, n_train, n_test, range1, range2, range3):
        # Prepare the training and test sets for the neural network solver
        self.inv_solver = ann
        n_data = n_train + n_test
        joint_space = np.hstack((np.random.uniform(range1[0], range1[1], size=(n_data, 1)),
                                  np.random.uniform(range2[0], range2[1], size=(n_data, 1)),
                                  np.random.uniform(range3[0], range3[1], size=(n_data, 1))))
        cartesian_space = np.zeros(np.shape(joint_space))
        for i in range(len(joint_space)):
            ax, ay, az = direct_kin_(joint_space[i], [self.link1, self.link2, self.link3])
            cartesian_space[i] = [ax[3], ay[3], az[3]]
        self.cart_train = np.asarray(cartesian_space[:n_train, :])
        self.joint_train = np.asarray(joint_space[:n_train, :])
        self.cart_test = np.asarray(cartesian_space[n_train:, :])
        self.joint_test = np.asarray(joint_space[n_train:, :])

    def train_inv_kin(self):
        # Train the kinematic solver
        self.inv_solver.fit(self.cart_train, self.joint_train)
        train_score = self.inv_solver.score(self.cart_train, self.joint_train)
        print(f"Training accuracy: {train_score:.4f}")
        return train_score

    def test_inv_kin(self):
        # Test the kinematic solver
        test_score = self.inv_solver.score(self.cart_test, self.joint_test)
        print(f"Testing accuracy: {test_score:.4f}")
        return test_score

    def inv_kin(self, Cartesian):
        # Query the trained inverse kinematic solver for a single Cartesian target
        joints = self.inv_solver.predict([Cartesian])
        [self.joint1, self.joint2, self.joint3] = joints[0]
        self.direct_kin()
        err = np.sqrt((Cartesian[0] - self.X[3])**2 + (Cartesian[1] - self.Y[3])**2 + (Cartesian[2] - self.Z[3])**2)
        return err, [self.X[3], self.Y[3], self.Z[3]]


## Example Usage


In [ ]:
# Example usage:
arm_instance = arm(links=[10, 10, 10], origin=[0, 0, 0], init=[0, 0, 0])
ann = MLPRegressor(hidden_layer_sizes=(200, 175, 150, 125, 100, 75, 50, 25), activation='relu', solver='adam',learning_rate='adaptive',learning_rate_init=0.0001, max_iter=1000)
n_train = 1000
n_test = 200
j1_range = (0, np.pi/2)
j2_range = (0, np.pi)
j3_range = (0, np.pi/2)
arm_instance.create_data(ann, n_train, n_test, j1_range, j2_range, j3_range)
train_accuracy = arm_instance.train_inv_kin()
test_accuracy = arm_instance.test_inv_kin()
